# Assignment 10

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

贝叶斯公式

P(c|x) = P(x,c)/P(x),等同于P(c|x) = P(c)P(x|c)/P(x),将求后验概率P(c|x)的问题转化为求类先验概率P(c),和求类条件概率P(x|c)。

根据大数定律，当数据集包含充足的独立同分布样本时，P(c)可以通过各类样本出现的频率进行估计。

但对于类条件概率来说，它涉及关于x所有属性的联合概率，但实际情况中，训练样本数往往低于样本空间的所有取值数，使得以样本频率估计P(c|x)不可行。因此，为简化求解。科学家假设所有的属性值互相独立

基于属性独立假设

P(c|x) = P(c)P(x|c)/P(x) = P(c)/p(x) ∏(i=1,d) P(xi|c)

#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

ML:
假设一个袋子装有白球与红球，比例未知，现在抽取10次（每次抽完都放回，保证事件独立性），假设抽到了7次白球和3次红球，在此数据样本条件下，可以采用最大似然估计法求解袋子中白球的比例（最大似然估计是一种“模型已定，参数未知”的方法）。

f(x1,x2|c) = f(x1|c)f(x2|c),其中c是未知的，因此我们定义似然L为

L(c|x1,x2) = f(x1,x2|c) = ∏(i=1,2) f(xi|c)

两边取ln，将右边等式的乘号转变为加号，方便求导。

lnL(c|x1,x2) = ln∑(i=1,2) f(xi|c) = ∑ln f(xi|c)

l（估计） = 1/2 lnL(c|x1,x2) 

求最大似然，即寻找一个合适的c，使得上式平均对数似然的值为最大

MAP:

根据贝叶斯理论，P(c|x) = P(c)P(x|c)/p(x),要使得后验概率P(c|x)为最大

即c（估计值）MAP = argmax P(c|x) = argmax P(c)P(x|c)

为了求得参数c，对P(x|c)求梯度，并使其等于0：

P(c|x)/∂c = P(c)P(x|c)/∂c = 0


#### 3. What is support vector in SVM?

KKT条件下推导：
1.ai >= 0
2. yi(wixi + b) - 1 >= 0
3. ai(yi(wixi + b) - 1) = 0

对于任意训练样本，总有ai=0或者yi（w*xi + b）=1。若ai=0，则样本不会在最后求解模型参数的式子中出现。若ai>0,则必有yi（w*xi + b）=1，所对应的样本位于最大间隔边界上，是一个支持向量。这显示SVM训练完成后，大部分训练样本都不需要保留，最终模型仅与支持向量有关。

#### 4. What is the intuition behind SVM ?

不是很理解这句话。SVM的直觉应该是把二分类问题放到几何上理解。

把二分类问题转化为在二维空间中，找到最佳分隔线。

这个分隔线怎么找才最好，最直觉的思想就是"不偏不倚"，与两类样本点的边界保持相等的最远的距离。

我们希望找到这样一个决策边界，这个边界距离两类数据点最远。更进一步的，是距离两类数据点的边界最远，所以定义最接近边界的数据点定义为支持向量。最后，我们的目标变为找到这样一个直线（多维叫超平面），它与支持向量有最大的间隔。

#### 5. Shortly describ what 'random' means in random forest ?

random forest使用了集成学习中的Bagging思想。Bagging算法具体过程如下。

1.从原始样本集中抽取训练集。每轮从原始样本集中使用Bootstraping方法抽取n个训练样本（放回可重复抽样）。共进行k轮抽取。

2.每一次使用一个训练集训练得到一个模型，k个训练集得到k个模型。

3对分类问题：将上步得到的k个模型用投票方式得到分类结果。

random体现在抽取训练集时采用放回可重复抽样。

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

XGboost中，我们构建新的树，希望损失函数尽可能小。

即，min∑（k=1，n）（L(yk,yk(估计)+Ft+1(Xk)） ,使用二阶泰勒展开可以得到

约为min∑（k=1，n）(GkFt+1(Xk) + 1/2Hk Ft+1^2(Xk)) + Ω（Ft+1）

而树模型fi是由多个叶子节点w1，w2，......,wTi组成

min泰勒展开近似可到w* = -∑(i∈Ij)Gi/∑(i∈Ij)Hi + λ

该点就是最优决策点

### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

## 数据获取

In [1]:
import pandas as pd
import numpy as np
import math
import jieba
import collections
import thulac
import pickle

In [2]:
filename = "C:\\NLP学习备用\\新华社数据.csv"
data = pd.read_csv(filename,encoding ="gb18030",engine = 'python')
data = data.dropna(subset=["content"]) #删除内容为空的行

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\NLP学习备用\\新华社数据.csv'

In [3]:
content = data["content"].tolist()
source = data["source"].tolist()

In [4]:
source = [name if isinstance(name,str) else "unknow" for name in source] #将没有来源的信息标记为unKnown

## content清洗切词，source分类标签

In [5]:
article_labels = [1 if name.strip()== "新华社" else 0 for name in source]

In [6]:
collections.Counter(article_labels)
#可以知道正负样本很不平衡，在训练模型时注意调整参数

Counter({0: 8393, 1: 78661})

In [24]:
import os

def get_stopwords():
    stopwords = []
    with open(r"c:\NLP学习备用\停用词表.txt",encoding="utf-8") as f:
        line_str = f.readline()
        while line_str != "":
            line_str = line_str.strip()
            stopwords.append(line_str)
            line_str = f.readline()
    return set(stopwords)

def text_cut(text_list):
    stopwords = get_stopwords()
    corpus = []
    count = 0
    str_all = ""
    for number,line in enumerate(text_list):
        sentence_sign = str(line) + "+sen" #用来记住每段的末尾
        str_all += sentence_sign
        
        if number % 1000 == 0 and number != 0:          
            thu1 = thulac.thulac(seg_only=True,filt=True) 
            words = thu1.cut(str_all, text=True)
            sentences = words.split(r"+ sen")
            print(number)
            str_all = ""
            corpus = corpus + sentences
            
        if number == len(text_list) - 1:           
            thu1 = thulac.thulac(seg_only=True,filt=True) 
            words = thu1.cut(str_all, text=True)
            sentences = words.split(r"+ sen")
            print(sentences)
            str_all = ""
            corpus = corpus + sentences
    return corpus

In [ ]:
corpus = text_cut(content)

In [25]:
def corpus_clean(corpus):
    clean_corpus = []
    stopwords = get_stopwords()
    for count,cor in enumerate(corpus):
        string = ""
        for word in cor:
            if word not in stopwords:
                attend_word = word + " "
                string = string + attend_word
        clean_corpus.append(string)
        print(string)
        if count % 1000 == 0 and count != 0: 
            print(count)
    return clean_corpus

In [ ]:
corpus_clean = corpus_clean(corpus)

In [ ]:
import pickle
with open('c:\NLP学习备用\corpus_list','wb') as f:
    pickle.dump(corpus,f)
with open('c:\NLP学习备用\corpus_clean_list','wb') as f:
    pickle.dump(corpus_clean,f)

In [2]:
import pickle

with open(r'c:\NLP学习备用\corpus_clean_list','rb') as f:
    corpus_clean = pickle.load(f)
with open(r'c:\NLP学习备用\corpus_list','rb') as f:
    corpus = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\NLP学习备用\\corpus_clean_list'

In [3]:
corpus

NameError: name 'corpus' is not defined

In [32]:
corpus_label = article_labels[:len(corpus_clean)]

## 用时测试

In [34]:
import time
def clock(func):
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        print('函数 {} 运行时间:{:.2f}s'.format(name,elapsed))
        return result
    return clocked

## 模型评估函数

In [35]:
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score

def get_performance(clf, x_, y_):
    y_hat = clf.predict(x_)
    
    print('f1_score is: {}'.format(f1_score(y_, y_hat)))
    print('accuracy is: {}'.format(accuracy_score(y_, y_hat)))
    print('percision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))


    return [format(f1_score(y_, y_hat),'.4f'),format(accuracy_score(y_, y_hat),'.4f'),
            format(f1_score(y_, y_hat),'.4f'),format(recall_score(y_, y_hat),'.4f')]


## 数据不均衡处理

In [36]:
from sklearn.model_selection import train_test_split
import random

corpus_0 = [a for a,i in zip(corpus_clean, corpus_label) if i==0]
corpus_1 = [a for a,i in zip(corpus_clean, corpus_label) if i==1]

print(len(corpus_0))
print(len(corpus_1))

X = corpus_0 + random.sample(corpus_1, len(corpus_0))
y = [0]*len(corpus_0) + [1]*len(corpus_0)

#后来想了想，这样简单过采样，是会造成数据泄露的，nlp数据增强要研究一下

print(len(X))
print(len(y))

8393
78634
16786
16786


## 文本向量化，划分数据集

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
vectorized = TfidfVectorizer(max_features= 5000) #设置文本单词个数最大值
X = vectorized.fit_transform(X)
X = X.toarray()

In [38]:
#选取30%左右数据作为测试数据
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
# import xgboost as xgb
import lightgbm as lgb

OSError: dlopen(/Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so, 6): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib
  Referenced from: /Users/junjiexie/opt/anaconda3/lib/python3.7/site-packages/lightgbm/lib_lightgbm.so
  Reason: image not found

In [40]:
def KNN(X_train, X_test, y_train, y_test,metric='minkowski'):
    nbrs = KNeighborsClassifier(n_neighbors=5, algorithm='brute',metric =metric)
    nbrs.fit(X_train, y_train)
    return get_performance(nbrs,X_test,y_test)

def random_forest(X_train, X_test, y_train, y_test,n_estimators=10,criterion='gini',max_depth=None):
    clf = RandomForestClassifier(n_estimators=n_estimators,criterion=criterion,max_depth=max_depth)
    clf.fit(X_train, y_train)
    return get_performance(clf,X_test,y_test)

def xgboost(X_train, X_test, y_train, y_test):
    clf = xgb.XGBRFClassifier(class_weight="balanced")
    clf.fit(X_train, y_train)
    return get_performance(clf,X_test,y_test)
def lightgbm(X_train, X_test, y_train, y_test):
    clf = lgb.LGBMClassifier(class_weight="balanced")
    clf.fit(X_train, y_train)
    return get_performance(clf,X_test,y_test)

In [41]:
KNN(X_train, X_test, y_train, y_test)

f1_score is: 0.7450495049504949
accuracy is: 0.7545671167593329
percision is: 0.7845351867940921
recall is: 0.7093479968578161
roc_auc is: 0.7550756048546109


['0.7450', '0.7546', '0.7450', '0.7093']

In [42]:
random_forest(X_train, X_test, y_train, y_test)

f1_score is: 0.9226644669003918
accuracy is: 0.9255361397934869
percision is: 0.9713417281806339
recall is: 0.8786331500392773
roc_auc is: 0.9260635629714458


['0.9227', '0.9255', '0.9227', '0.8786']

In [43]:
xgboost(X_train, X_test, y_train, y_test)

f1_score is: 0.9672447013487476
accuracy is: 0.9662430500397141
percision is: 0.9493192133131618
recall is: 0.98586017282011
roc_auc is: 0.9660224558879666


C:\Users\junjiexie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['0.9672', '0.9662', '0.9672', '0.9859']

In [44]:
lightgbm(X_train, X_test, y_train, y_test)

f1_score is: 0.9747081712062257
accuracy is: 0.9741858617950755
percision is: 0.9656900539707016
recall is: 0.9838963079340142
roc_auc is: 0.974076668023232


C:\Users\junjiexie\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


['0.9747', '0.9742', '0.9747', '0.9839']

总的来说，集成学习梯度提升树模型表现是最好的。lightgbm据说效果比xgboost好，好像也的确如此。但实战中发现，lightgbm好像比较容易过拟合，catboost没有试过。但在数据科学竞赛中，三大提升树表现确实是机器学习模型中最好的

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).